# **1. INTRODUCTION**

* Link prediction (Predicting the interactions between nodes) là một nhiệm vụ quan trọng trong lĩnh vực học máy đồ thị. Với sự tồn tại phổ biến của đồ thị, một nhiệm vụ như vậy có nhiều ứng dụng, chẳng hạn như:
    * Recommender system 
    * Knowledge graph completion 
    * Drug repurposing 

* Các phương pháp dự đoán liên kết truyền thống thường xác định các heuristic metrics khác nhau trên các đường dẫn giữa một cặp nút:
    * Katz index [30] được định nghĩa là số lượng đường dẫn có trọng số giữa hai nút.
    * Personalized PageRank [42] đo lường mức độ giống nhau của hai nút dưới dạng random walk probability từ nút này sang nút khác
    * Graph distance [37] sử dụng độ dài đường đi ngắn nhất giữa hai nút để dự đoán mối liên hệ của chúng

* Các phương pháp này có thể được áp dụng trực tiếp cho các biểu đồ mới (inductive setting), có khả năng diễn giải tốt và mở rộng quy mô thành các biểu đồ lớn. Tuy nhiên, chúng được thiết kế dựa trên các metrics thủ công và **có thể không tối ưu cho việc dự đoán liên kết trên biểu đồ trong thế giới thực**

* Để giải quyết những hạn chế này, một số phương pháp dự đoán liên kết áp dụng graph neural networks (GNN) để tự động trích xuất các tính năng quan trọng từ các vùng lân cận cục bộ để dự đoán liên kết. Nhờ tính expressiveness cao của GNN, các phương pháp này đã cho thấy hiệu suất vượt trội. Tuy nhiên, **những phương pháp này chỉ có thể được áp dụng để dự đoán các liên kết mới trên biểu đồ huấn luyện (transductive setting) và thiếu khả năng diễn giải**. Mặc dù một số phương pháp gần đây trích xuất các tính năng từ các sơ đồ con cục bộ bằng GNN và hỗ trợ cài đặt quy nạp, nhưng khả năng mở rộng của các phương pháp này bị ảnh hưởng.
    * Induction là suy luận từ các trường hợp training được quan sát đến các quy tắc chung, sau đó được áp dụng cho các trường hợp kiểm thử.
    * Transduction là suy luận từ các trường hợp (training) cụ thể được quan sát đến các trường hợp (test) cụ thể.

* Do đó, chúng tôi tự hỏi liệu có tồn tại một cách tiếp cận tận dụng được các ưu điểm của cả phương pháp dựa trên traditional path-based và các phương pháp tiếp cận gần đây dựa trên graph neural networks, tức là khái quát hóa trong cài đặt inductive, khả năng diễn giải, công suất mô hình cao và khả năng mở rộng.

* **Giải pháp**: Lấy cảm hứng từ các phương pháp dựa trên traditional path-based, mục tiêu của chúng tôi là phát triển learning framework thông thường và linh hoạt để dự đoán liên kết dựa trên đường dẫn giữa hai nút. Cụ thể, chúng tôi xác định cách biểu diễn của một cặp nút là tổng tổng quát của tất cả các biểu diễn đường dẫn giữa chúng, trong đó mỗi biểu diễn đường dẫn được xác định là tích tổng quát của các biểu diễn cạnh trong đường dẫn.
    * Katz index [30] (link prediction method)
    * Personalized PageRank [42] (link prediction method)
    * Graph distance [37] (link prediction method)
    * Widest path [4] (graph theory algorithms)
    * Most reliable path [4] (graph theory algorithms) 
    
    là những trường hợp đặc biệt của công thức đường dẫn này với các toán tử tính tổng và nhân khác nhau. Được thúc đẩy bởi thuật toán thời gian đa thức cho bài toán đường đi ngắn nhất [5], chúng tôi cho thấy rằng công thức như vậy có thể được giải quyết một cách hiệu quả thông qua thuật toán Bellman-Ford tổng quát [4] trong điều kiện nhẹ và mở rộng lên các đồ thị lớn.

* Các toán tử trong thuật toán Bellman-Ford tổng quát —summation and multiplication— được thực hiện thủ công, **có tính linh hoạt hạn chế**. Do đó, chúng tôi đề xuất thêm Mạng Neural Bellman-Ford (NBFNet), một khung mạng nơ-ron đồ thị giải quyết công thức đường dẫn trên với các toán tử đã học trong thuật toán Bellman-Ford tổng quát. Cụ thể, NBFNet tham số hóa thuật toán Bellman-Ford tổng quát với ba thành phần thần kinh là các hàm INDICATOR, MESSAGE và AGGREGATE.
    * Hàm INDICATOR khởi tạo một biểu diễn trên mỗi nút, được lấy làm điều kiện biên của thuật toán Bellman-Ford tổng quát.
    * Hàm MESSAGE học các toán tử nhân tương ứng
    * hàm AGGREGATE tìm hiểu các toán tử tính tổng

* Ta chứng minh rằng:
    * Hàm MESSAGE có thể được định nghĩa theo các toán tử quan hệ trong các phần nhúng biểu đồ tri thức [6, 68, 58, 31, 52], ví dụ như một bản dịch trong không gian Euclide do các toán tử quan hệ của TransE [6] tạo ra.
    * Hàm AGGREGATE có thể được định nghĩa là hàm tổng hợp tập hợp có thể học được [71, 65, 9].
    
    Với việc tham số hóa như vậy, NBFNet có thể khái quát hóa cài đặt inductive, đồng thời đạt được một trong những độ phức tạp về thời gian thấp nhất trong số các phương pháp inductive GNN. Hiển thị Bảng 1. Với các phiên bản khác của hàm MESSAGE và AGGREGATE, khung của chúng tôi cũng có thể khôi phục một số công việc hiện có về các quy tắc logic học [69, 46] để dự đoán liên kết trên biểu đồ tri thức (Bảng 2).

* Khung NBFNet của chúng tôi có thể được áp dụng cho một số biến thể dự đoán liên kết, không chỉ bao gồm các single-relational graphs (e.g., homogeneous graphs) mà còn cả các multi-relational graphs (e.g., knowledge graphs). Chúng tôi đánh giá thực nghiệm NBFNet được đề xuất để dự đoán liên kết trên các biểu đồ đồng nhất và biểu đồ tri thức trong cả cài đặt chuyển nạp và quy nạp. Kết quả thử nghiệm cho thấy NBFNet được đề xuất vượt trội hơn các phương pháp tiên tiến hiện có với mức chênh lệch lớn trong tất cả các cài đặt, với mức tăng hiệu suất tương đối trung bình là 18% ở việc hoàn thành knowledge graph (HITS@1) và 22% khi inductive relation prediction (HITS@10). Chúng tôi cũng chỉ ra rằng NBFNet được đề xuất thực sự có thể hiểu được bằng cách trực quan hóa các đường dẫn có liên quan hàng đầu để dự đoán liên kết trên knowledge graphs.

# **2. RELATED WORK**

Công việc hiện tại về link prediction thường có thể được phân loại thành 3 mô hình chính: phương pháp dựa trên đường dẫn, phương pháp nhúng và mạng lưới thần kinh đồ thị.

## 2.1 Path-based methods

* **HOMOGENEOUS GRAPH:**
    * Các phương pháp ban đầu trên homogeneous graphs tính toán độ giống nhau giữa hai nút dựa trên số lượng đường dẫn có trọng số (chỉ số Katz [30]), xác suất bước đi ngẫu nhiên (PageRank được cá nhân hóa [42]) hoặc độ dài của đường đi ngắn nhất (khoảng cách biểu đồ [37]).

    * SimRank [28] sử dụng các metrics nâng cao như khoảng cách gặp nhau dự kiến trên các homogeneous graphs, được PathSim [51] mở rộng sang các homogeneous graphs.
* **KNOWLEDGE GRAPH:**
    * Trên knowledge graphs, Path Ranking [35, 15] trực tiếp sử dụng các đường dẫn quan hệ làm đặc điểm tượng trưng cho dự đoán

* Phương pháp Rule mining, như là NeuralLP [69] and DRUM [46], tìm hiểu các quy tắc logic xác suất để cân nhắc các đường dẫn khác nhau.

* Phương pháp Path representation, như là Path-RNN [40] và các kế nhiệm của nó [11, 62], mã hóa từng đường dẫn bằng mạng thần kinh tái phát (RNN) và các đường dẫn tổng hợp để dự đoán.

* Tuy nhiên, **các phương thức này cần đi qua số lượng đường dẫn theo cấp số nhân và bị giới hạn ở các đường dẫn rất ngắn** ($<= 3$ edges).

* Để mở rộng quy mô cho path-based methods, All-Paths [57] đề xuất tổng hợp hiệu quả tất cả các đường dẫn bằng lập trình động. Tuy nhiên, **All-Paths bị hạn chế ở bilinear models và có dư lượng model hạn chế**. 

* Một luồng công việc khác [64, 10, 22] học một tác nhân để thu thập các đường dẫn hữu ích cho dự đoán liên kết. Mặc dù các phương pháp này có thể tạo ra các đường dẫn có thể giải thích được nhưng chúng có phần thưởng cực kỳ thưa thớt và yêu cầu kỹ thuật cẩn thận về chức năng phần thưởng [38] hoặc chiến lược tìm kiếm [50]. Một số công trình khác [8, 44] áp dụng suy luận biến phân để tìm hiểu trình tìm đường dẫn và trình suy luận đường dẫn để dự đoán liên kết.

## 2.2 Embedding Methods

* Phương pháp Embedding tìm hiểu cách biểu diễn phân tán cho từng nút và cạnh bằng cách bảo toàn cấu trúc cạnh của biểu đồ. 

* Phương pháp biểu diễn bao gồm:
    * DeepWalk [43] and LINE [53] on homogeneous graphs
    * TransE [6], DistMult [68] and RotatE [52] on knowledge graphs. 

* Các công trình sau này cải tiến các phương pháp nhúng với các hàm điểm mới [58, 13, 31, 52, 54, 76] nắm bắt các mẫu ngữ nghĩa chung của các mối quan hệ hoặc tìm kiếm hàm điểm trong không gian thiết kế chung [75]. 

* Phương pháp Embedding đạt được kết quả đầy hứa hẹn về dự đoán liên kết và có thể được chia tỷ lệ thành các biểu đồ rất lớn bằng cách sử dụng nhiều GPU [78].

* Tuy nhiên, **phương pháp Embedding không mã hóa rõ ràng các sơ đồ con cục bộ giữa các cặp nút và không thể áp dụng cho cài đặt inductive**

## 2.3 Graph Neural Network

* Graph neural networks (GNNs) [47, 33, 60, 65] là một họ các mô hình học biểu diễn mã hóa các cấu trúc topology của đồ thị. 
    * **Về link prediction**: 
        * Các khung phổ biến [32, 48, 12, 59] áp dụng công thức mã hóa tự động, sử dụng GNN để mã hóa các biểu diễn nút và giải mã các cạnh dưới dạng hàm trên các cặp nút. Các frameworks như vậy có khả năng inductive nếu tập dữ liệu cung cấp các tính năng nút, nhưng chỉ có tính chất transductive khi các tính năng nút không có sẵn.
        * Một số frameworks khác, chẳng hạn như SEAL [73] và GraIL [55], mã hóa rõ ràng sơ đồ con xung quanh mỗi cặp nút để dự đoán liên kết. Mặc dù các framework này được chứng minh là mạnh hơn công thức mã hóa tự động [74] và có thể giải quyết cài đặt inductive, nhưng chúng yêu cầu cụ thể hóa một sơ đồ con cho mỗi liên kết, không thể mở rộng thành các biểu đồ lớn.

* Ngược lại, NBFNet nắm bắt rõ ràng các đường dẫn giữa hai nút để dự đoán liên kết, đồng thời đạt được độ phức tạp về thời gian tương đối thấp (Bảng 1).

* ID-GNN [70] chính thức hóa dự đoán liên kết như một nhiệm vụ phân loại nút có điều kiện và tăng cường GNN bằng danh tính của nút nguồn. Mặc dù kiến trúc của NBFNet có chung một số tinh thần với ID-GNN, nhưng mô hình (NBFNet) được thúc đẩy bởi thuật toán Bellman-Ford tổng quát và có các kết nối lý thuyết với các phương pháp dựa trên path-based truyền thống.

* Ngoài ra còn có một số công việc đang cố gắng mở rộng quy mô GNN để dự đoán liên kết bằng cách cắt bớt động tập hợp các nút trong quá trình truyền thông báo [66, 20].

* Các phương pháp này bổ sung cho NBFNet và có thể được tích hợp vào phương pháp để cải thiện hơn nữa khả năng mở rộng.

# **3. METHODOLOGY**

## 3.1 Path formulation for LINK PREDICTION

**A knowledge graph** is denoted by G = (V , E , R), where V and E represent the set of entities (nodes) and relations (edges) respectively, and R is the set of relation types. We use N(u) to denote the set of nodes connected to u, and E(u) to denote the set of edges ending with node u. 

**A homogeneous graph** G = (V , E) can be viewed as a special case of knowledge graphs, with only one relation type for all edges. 

Throughout this paper, we use bold terms, $w_q(e)$ or $h_q(u, v)$, to denote vector representations, and italic terms, $w_e$ or $w_{uv}$ , to denote scalars like the weight of edge (u, v) in homogeneous graphs or triplet (u, r, v) in knowledge graphs. Without loss of generality, we derive our method based on knowledge graphs, while our method can also be applied to homogeneous graphs.

### **PATH FORMRULATION**

**Link prediction** is aimed at predicting the existence of a query relation q between a head entity u and a tail entity v. From a representation learning perspective, this requires to learn a pair representation $h_q(u,v)$, which captures the local subgraph structure between u and v With Regard To the query relation q. 

In traditional methods, such a local structure is encoded by counting different types of random walks from u to v [35, 15]. Inspired by this construction, we formulate the pair representation as a generalized sum of path representations between u and v with a commutative summation operator $⊕$. Each path representation $h_q(P)$ is defined as a generalized product of the edge representations in the path with the multiplication operator $⊗$.

![path formulation](./Images/pathformulation.png "Path Formulation")

where $P_{uv} denotes the set of paths from u to v and $w_q(e_i)$ is the representation of edge e_i. Note the multiplication operator ⊗ is not required to be commutative (e.g., matrix multiplication), therefore we define ⊗ to compute the product following the exact order. 

Intuitively, the path formulation can be interpreted as a depth-first-search (DFS) algorithm, where one searches all possible paths from u to v, computes their representations (Equation 2) and aggregates the results (Equation 1). Such a formulation is capable of modeling several traditional link prediction methods, as well as graph theory algorithms. 

Formally, Theorem 1-5 state the corresponding path formulations for 3 link prediction methods and 2 graph theory algorithms respectively. See Appendix A for proofs.

* **Theorem 1** Katz index is a path formulation with ⊕ = +, ⊗ = × and $w_q(e) = \beta w_e$.

* **Theorem 2** Personalized PageRank is a path formulation with ⊕ = +, ⊗ = × and $w_q(e) = \alpha w_{uv} / \sum_{}{v′\in N(u)}w_{uv′}$

* **Theorem 3** Graph distance is a path formulation with ⊕ = min, ⊗ = + and $w_q(e) = w_e$ 

* **Theorem 4** Widest path is a path formulation with ⊕ = max, ⊗ = min and $w_q(e) = w_e$ 

* **Theorem 5** Most reliable path is a path formulation with ⊕ = max, ⊗ = × and $w_q(e) = w_e$

### **Generalized Bellman-Ford Algorithm**

While the above formulation is able to model important heuristics for link prediction, it is computationally expensive since the number of paths grows exponentially with the path length.

Previous works [40, 11, 62] that directly computes the exponential number of paths can only afford a maximal path length of 3. 

A more scalable solution is to use the generalized Bellman-Ford algorithm [4]. Specifically, assuming the operators ⟨⊕, ⊗⟩ satisfy a semiring system [21] with summation identity $0_q$ and multiplication identity $1_q$, we have the following algorithm.

![Generalized Bell-man Ford Algorithm](./Images/generalizedbellmanfordalgorithm.png "Generalized Bell-man Ford Algorithm")

* where $1_q(u = v)$ is the indicator function that outputs $1_q$ if u = v and $0_q$ otherwise. * $w_q(x, r, v)$ is the representation for edge $e = (x, r, v)$
* $r$ is the relation type of the edge. 
* Equation 3 is known as the boundary condition
* Equation 4 is known as the Bellman-Ford iteration. 

The high-level idea of the generalized Bellman-Ford algorithm is to compute the pair representation $h_q(u,v)$ for a given entity u, a given query relation q and all $v \in V$ in parallel, and reduce the total computation by the distributive property of multiplication over summation. 

Since u and q are fixed in the generalized Bellman-Ford algorithm, we may abbreviate $h_q^{(t)}(u, v)$ as $h_v^{(t)}$ when the context is clear. When ⊕ = min and ⊗ = +, it recovers the original Bellman-Ford algorithm for the shortest path problem [5]. See Appendix B for preliminaries and the proof of the above algorithm.

* **Theorem 6** Katz index, personalized PageRank, graph distance, widest path and most reliable path can be solved via the generalized Bellman-Ford algorithm.